In [1]:
import os
import numpy as np
import librosa
import tensorflow as tf
from sklearn.model_selection import train_test_split
from model import EnvNet
from train import train_model
from data_preprocess import make_frames,make_frames_folder
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv1D,MaxPooling1D,Dense,Flatten,BatchNormalization,Dropout, Activation
from gammatone_init import GammatoneInit
from gammatone_init import generate_filters
from model_config import *
import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import ast
import wandb


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
frame_length = 16000
overlapping_fraction = 0.5
data = torch.load('./torch_dataset_16khz/all_audio_data.pt')
print(data.size())

torch.Size([54033, 16001])


In [4]:
def to_categorical(tensors, num_classes=10):
    return torch.eye(num_classes)[y.int()]
print(data.size())
tensor_size = (data.size())[0]
print(tensor_size)

torch.Size([54033, 16001])
54033


In [5]:
X_train = data[:,0:frame_length].clone()
test_portion = int(0.8*((data.size())[0]))
print(test_portion)
X_train = data[:test_portion, 0:frame_length].clone()
print(X_train.size())
Y_train = data[:test_portion,frame_length:].clone()

X_train = X_train.reshape(-1,16,1000)

print(X_train.size())
print(Y_train.size())
print(Y_train)

43226
torch.Size([43226, 16000])
torch.Size([43226, 16, 1000])
torch.Size([43226, 1])
tensor([[3.],
        [3.],
        [3.],
        ...,
        [2.],
        [2.],
        [2.]], dtype=torch.float64)


In [17]:
Y_train = Y_train.type(torch.LongTensor)
Y_train_one_hot = F.one_hot(Y_train)
#print(Y_train_one_hot)
print(Y_train)
audio_dataset = TensorDataset (X_train, Y_train)
audio_dataloader = DataLoader (audio_dataset, batch_size = 100, shuffle= True)

tensor([[3],
        [3],
        [3],
        ...,
        [2],
        [2],
        [2]])


In [7]:
sample_rate = 16000
min_center_freq = 100
order = 2

In [8]:
class RMSLELoss(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = torch.nn.MSELoss()
        
    def forward(self, pred, actual):
        return self.mse(torch.log(pred.float() + 1), torch.log(actual.float() + 1))
 

In [15]:
loss_function = torch.nn.CrossEntropyLoss()
input = torch.randn(100, 10, requires_grad=True)
target = torch.empty(100, dtype=torch.long).random_(5)
print(input.size())
print(target.size())


torch.Size([100, 10])
torch.Size([100])


In [20]:
from featureblocks import FeatureBlock3
from featureblocks import FeatureBlockGT

# The trinity of models
model = FeatureBlock3()
#model = FeatureBlockGT()
# This is the losss function
#loss_function = RMSLELoss()
loss_function = torch.nn.CrossEntropyLoss()
# This is what controls the gradient descent
#optimizer = torch.optim.Adadelta(model.parameters(),lr=1)
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
iteration = 0
losses = []

for epoch in range(100):
    
    for index,(x,y) in enumerate(audio_dataloader):
        optimizer.zero_grad()
        #print(x.size())
        x = model(x.float())
        #print(x.size())
        #print(y.size())
        # Use argmax to get class with max probability value from softmax
        #x = x.argmax(dim=-1) 
        x = x.float()
        y = y.squeeze(1)
        
        loss = loss_function(x,y)
        loss.backward()
        optimizer.step()
        print("iteration:{} loss:{} ".format(iteration, loss.item()))
        losses.append(loss)
        iteration += 1


iteration:0 loss:2.306471824645996 
iteration:1 loss:2.3028924465179443 
iteration:2 loss:2.300689458847046 
iteration:3 loss:2.302952527999878 
iteration:4 loss:2.2994871139526367 
iteration:5 loss:2.3021552562713623 
iteration:6 loss:2.2985341548919678 
iteration:7 loss:2.2980194091796875 
iteration:8 loss:2.2954187393188477 
iteration:9 loss:2.2970099449157715 
iteration:10 loss:2.2973315715789795 
iteration:11 loss:2.2904937267303467 
iteration:12 loss:2.2926065921783447 
iteration:13 loss:2.2910759449005127 
iteration:14 loss:2.289609432220459 
iteration:15 loss:2.28837513923645 
iteration:16 loss:2.282341957092285 
iteration:17 loss:2.283125400543213 
iteration:18 loss:2.2791924476623535 
iteration:19 loss:2.2787892818450928 
iteration:20 loss:2.2827463150024414 
iteration:21 loss:2.261875629425049 
iteration:22 loss:2.275815010070801 
iteration:23 loss:2.2833023071289062 
iteration:24 loss:2.2808725833892822 
iteration:25 loss:2.2615749835968018 
iteration:26 loss:2.247562170028

KeyboardInterrupt: 

In [76]:
shape = [16,16,64]
filters = generate_filters(shape[2],shape[0],sample_rate,min_center_freq,order)
filters = filters.reshape(filters.shape[1],1,filters.shape[0])


In [1]:
import matplotlib.pyplot as plt
y = np.array(losses, dtype=float)
x = np.arange(len(losses))
print(x.dtype)
print(y.dtype)

m, b = np.polyfit(x, y, 1)
plt.plot(x, y, 'o')
plt.plot(x, m*x+b)

NameError: name 'np' is not defined